In [19]:
%pip install pandas scikit-learn
import pandas as pd
from sklearn.preprocessing import StandardScaler

Note: you may need to restart the kernel to use updated packages.


## Limpieza y Preprocesamiento de Datos de Aguacate

Este cuaderno contiene los pasos seguidos para limpiar y preparar los datos de aguacate para su posterior análisis y uso en modelos de machine learning.

### 1. Cargar y manejar los valores nulos

El primer paso consiste en cargar el conjunto de datos y manejar los valores nulos. Para ello, se utilizan las librerías de `pandas` para cargar el archivo CSV y luego se manejan los valores nulos.

In [20]:
avocado_data = pd.read_csv("./data/avocado.csv")

print('Valores nulos a processar:')
print(avocado_data.isnull().sum())

num_cols = avocado_data.select_dtypes(include='number').columns
avocado_data[num_cols] = avocado_data[num_cols].fillna(avocado_data[num_cols].mean())

cat_cols = avocado_data.select_dtypes(include='object').columns
for col in cat_cols:
    avocado_data[col] = avocado_data[col].fillna(avocado_data[col].mode()[0])

Valores nulos a processar:
Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64


#### Explicación:
- Se cargan los datos desde el archivo CSV.
- Para las columnas numéricas, se sustituyen los valores nulos por la media de cada columna. Esto es una estrategia común cuando se trata con valores nulos en variables numéricas, ya que ayuda a mantener la distribución general de los datos.
- Para las columnas categóricas, se sustituyen los valores nulos por el valor más frecuente (moda) de cada columna. Esto es útil cuando se tienen variables como categorías, donde reemplazar valores nulos por el valor más común es una opción válida.

### 2. Eliminar columnas innecesarias
A continuación, se elimina una columna que no aporta información relevante al análisis o modelo

In [21]:
avocado_data.drop(columns=['Unnamed: 0'], inplace=True)

#### Explicación:
Se elimina la columna 'Unnamed: 0' porque  no contiene información útil para el análisis o los modelos.

### 3. Detección y tratamiento de valores atípicos
El siguiente paso es detectar y tratar los valores atípicos. Para esto, se utiliza el método de Rango Intercuartílico (IQR) para identificar los valores que se encuentran fuera de los límites establecidos.

In [22]:
cols_outliers = ['AveragePrice', 'Total Volume', '4046', '4225', '4770']
for col in cols_outliers:
    Q1 = avocado_data[col].quantile(0.25)
    Q3 = avocado_data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    avocado_data = avocado_data[(avocado_data[col] >= lower) & (avocado_data[col] <= upper)]

#### Explicación:

- Se seleccionan las columnas numéricas relevantes que pueden contener valores atípicos.
- Para cada columna, se calculan los valores de los cuartiles (Q1 y Q3) y el Rango Intercuartílico (IQR). Los valores atípicos se identifican como aquellos que se encuentran fuera de los límites lower y upper (1.5 veces el IQR por debajo de Q1 y por encima de Q3).
- Luego, se filtra el conjunto de datos para eliminar las filas que contienen valores fuera de estos límites. Esto ayuda a mejorar la calidad de los datos y la precisión de los modelos al reducir la influencia de los valores extremos.

### 4. Conversión de variables categóricas a variables dummy
Después, se convierten las variables categóricas, como type y region, en variables numéricas utilizando One-Hot Encoding. Este proceso crea una nueva columna binaria para cada categoría dentro de una variable.

In [23]:
avocado_data = pd.get_dummies(avocado_data, columns=['type', 'region'], drop_first=True)

#### Explicación:
- Se utiliza la función pd.get_dummies para convertir las columnas categóricas type y region en columnas binarias. Cada categoría se convierte en una columna separada, donde un valor de 1 indica que la fila pertenece a esa categoría, y un valor de 0 indica que no.
- El parámetro drop_first=True se utiliza para evitar la multicolinealidad, es decir, se elimina una de las columnas generadas para cada variable categórica, ya que esta puede ser inferida a partir de las otras.

### 5. Estandarización de datos
En este paso, se estandarizan las características numéricas del conjunto de datos. La estandarización es crucial para los modelos de machine learning, ya que garantiza que todas las variables tengan la misma escala y ninguna domine sobre las demás debido a diferencias de magnitudes.

In [24]:

scaler = StandardScaler()
cols_to_scale = ['AveragePrice', 'Total Volume', '4046', '4225', '4770', 
                 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

avocado_data[cols_to_scale] = scaler.fit_transform(avocado_data[cols_to_scale])

#### Explicación:
- Se utiliza el StandardScaler de la librería sklearn para estandarizar las columnas numéricas seleccionadas. Este transformador asegura que todas las variables tengan media 0 y desviación estándar 1.
- La estandarización es fundamental cuando se usan modelos que son sensibles a las escalas de las variables, como los modelos basados en distancia (p. ej., KNN, SVM) o los que utilizan gradientes (p. ej., regresión logística, redes neuronales).

### 6. Guardar los datos limpios
Finalmente, se guarda el conjunto de datos limpio en un nuevo archivo CSV para su uso posterior.

In [25]:
avocado_data.to_csv("./data/avocado_cleaned.csv", index=False)

In [26]:
print(avocado_data.head())

         Date  AveragePrice  Total Volume      4046      4225      4770  \
0  2015-12-27     -0.678662      0.208861 -0.402076  1.549884 -0.234086   
1  2015-12-20     -0.621101      0.078051 -0.416538  1.158791 -0.185652   
2  2015-12-13     -1.829879      0.963330 -0.411733  3.729048  0.158050   
3  2015-12-06     -1.398173      0.415083 -0.398275  2.247982 -0.117788   
4  2015-11-29     -0.822564      0.024420 -0.405877  1.126900 -0.102548   

   Total Bags  Small Bags  Large Bags  XLarge Bags  ...  region_Seattle  \
0   -0.384125   -0.277729   -0.363378    -0.144263  ...           False   
1   -0.361659   -0.248597   -0.363123    -0.144263  ...           False   
2   -0.399445   -0.298060   -0.362782    -0.144263  ...           False   
3   -0.464288   -0.383700   -0.360937    -0.144263  ...           False   
4   -0.453932   -0.372515   -0.357084    -0.144263  ...           False   

   region_SouthCarolina  region_SouthCentral  region_Southeast  \
0                 False         